<h1><center>Лабораторна робота 4.</center></h1>
<h2><center>Виявлення сарказму за допомогою логістичної регресії</center></h2>

**Виконав:** Прізвище І.П.

**Варіант:** №__

В цій лабораторній роботі ми будемо використовувати набір даних з [наукової статі](https://arxiv.org/abs/1704.05579) «A Large Self-Annotated Corpus for Sarcasm» з понад 1 мільйоном коментарів із Reddit, що позначені як саркастичні або ні. Оброблену версію цього набору даних можна знайти на Kaggle у формі [Набору даних Kaggle](https://www.kaggle.com/datasets/danofer/sarcasm/data).

<a class="anchor" id="lab-4"></a>

## Зміст

- [4.1. Попередній аналіз даних](#lab-4.1)
- [4.2. Векторизація TF-IDF та логістична регресія](#lab-4.2)
- [4.3. Інтерпретація та порівняння моделі](#lab-4.3)
- [4.4. Розширене вдосконалення моделі](#lab-4.4)
- [4.5. Практичне застосування результатів інтелектуального аналізу даних](#lab-4.5)

Спершу завантажте цільовий набір даних `train-balanced-sarcasm.csv` із ресурсу Kaggle за [посиланням](https://www.kaggle.com/datasets/danofer/sarcasm/data?select=train-balanced-sarcasm.csv). Збережіть файл .csv в теку з робочим файлом лабораторної роботи. 

In [1]:
PATH_TO_DATA = "train-balanced-sarcasm.csv"

In [2]:
# Виконаємо основні завантаження бібліотеки
import os

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

In [3]:
train_df = pd.read_csv(PATH_TO_DATA)

In [4]:
train_df.head()

,label,comment,author,subreddit,score,ups,downs,date,created_utc,parent_comment
0,0,NC and NH.,Trumpbart,politics,2,-1,-1,2016-10,2016-10-16 23:55:23,"Yeah, I get that argument. At this point, I'd ..."
1,0,You do know west teams play against west teams...,Shbshb906,nba,-4,-1,-1,2016-11,2016-11-01 00:24:10,The blazers and Mavericks (The wests 5 and 6 s...
2,0,"They were underdogs earlier today, but since G...",Creepeth,nfl,3,3,0,2016-09,2016-09-22 21:45:37,They're favored to win.
3,0,"This meme isn't funny none of the ""new york ni...",icebrotha,BlackPeopleTwitter,-8,-1,-1,2016-10,2016-10-18 21:03:47,deadass don't kill my buzz
4,0,I could use one of those tools.,cush2push,MaddenUltimateTeam,6,-1,-1,2016-12,2016-12-30 17:00:13,Yep can confirm I saw the tool they use for th...


In [5]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1010826 entries, 0 to 1010825
Data columns (total 10 columns):
 #   Column          Non-Null Count    Dtype 
---  ------          --------------    ----- 
 0   label           1010826 non-null  int64 
 1   comment         1010773 non-null  object
 2   author          1010826 non-null  object
 3   subreddit       1010826 non-null  object
 4   score           1010826 non-null  int64 
 5   ups             1010826 non-null  int64 
 6   downs           1010826 non-null  int64 
 7   date            1010826 non-null  object
 8   created_utc     1010826 non-null  object
 9   parent_comment  1010826 non-null  object
dtypes: int64(4), object(6)
memory usage: 77.1+ MB


Деяких коментарів бракує в наборі, тому ми видаляємо відповідні рядки.

In [6]:
train_df.dropna(subset=["comment"], inplace=True)

Переконаємось, що набір даних є справді збалансованим.

In [7]:
train_df["label"].value_counts()

0    505405
1    505368
Name: label, dtype: int64

Розділимо дані на частини для навчання та валідації.

In [8]:
train_texts, valid_texts, y_train, y_valid = train_test_split(
    train_df["comment"], train_df["label"], random_state=17
)

<a class="anchor" id="lab-4.1"></a>

## <span style="color:blue; font-size:1.2em;">4.1. Попередній аналіз даних</span>

[Повернутися до змісту](#lab-4)

In [9]:
from wordcloud import STOPWORDS, WordCloud

In [10]:
# Підготуємо об'єкт класу WordCloud для 
# майбутнього візуального аналізу  
my_wordcloud = WordCloud(
    background_color="black",
    stopwords=STOPWORDS,
    max_words=200,
    max_font_size=100,
    random_state=17,
    width=800,
    height=400,
)

<span style="color:red; font-size:2em;">Завдання 1</span>

**Для всіх варіантів:**

В Завданні 1 вам потрібно проаналізувати та підготувати набір даних для подальшої побудови моделей інтелектуального аналізу даних.

**Для варіантів 1-2:**

Проаналізуйте використання в текстах хештегів, URLs, емоджі та згадок користувачів (символ - @).

**Для варіантів 3-4:**

Токенізуйте текст на слова. Обрахуйте та візуалізуйте 20 найбільш вживаних слів.

**Для варіантів 5-6:**

Візуалізуйте розподіли саркастичних і нормальних коментарів.

**Для варіантів 7-8:**

Проаналізуйте, чи є деякі субредити в середньому більш «саркастичними», ніж інші.

**Для варіантів 9-10:**

Візуалізуйте хмарне подання слів для кожного класу.

**Для варіантів 11-12:**

Видаліть загальні стоп-слова з текстових даних.

**Для варіантів 13-14:**

Дослідіть та візуалізуйте розподіли довжини тексту для різних класів.

**Для варіантів 15-16:**

Напишіть функцію для очищення текстових даних (видалення спеціальних символів, нижнього регістру тощо).

**Для варіантів 17-18:**

Створіть хмару слів для найуживаніших слів у класі "Саркастичний".

**Для варіантів 19-20:**

Проаналізуйте, чи є деякі автори в середньому більш саркастичними, ніж інші.


<a class="anchor" id="lab-4.2"></a>

## <span style="color:blue; font-size:1.2em;">4.2. Векторизація TF-IDF та логістична регресія</span>

[Повернутися до змісту](#lab-4)

In [11]:
# build bigrams, put a limit on maximal number of features
# and minimal word frequency
# Побудуємо біграми, обмежимо максимальну кількість ознак
# і мінімальну частоту слів
tf_idf = TfidfVectorizer(ngram_range=(1, 2), max_features=50000, min_df=2)
# multinomial logistic regression a.k.a softmax classifier
# Створимо об'єкт мультиноміальної логістичної регресії,
# що також відома як класифікатор softmax
logit = LogisticRegression(C=1, n_jobs=4, solver="lbfgs", random_state=17, verbose=1)
# Задамо sklearn's pipeline
tfidf_logit_pipeline = Pipeline([("tf_idf", tf_idf), ("logit", logit)])

In [12]:
%%time
tfidf_logit_pipeline.fit(train_texts, y_train)

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.


CPU times: total: 18.4 s
Wall time: 28.3 s


Pipeline(steps=[('tf_idf',
                 TfidfVectorizer(max_features=50000, min_df=2,
                                 ngram_range=(1, 2))),
                ('logit',
                 LogisticRegression(C=1, n_jobs=4, random_state=17,
                                    verbose=1))])

In [13]:
%%time
valid_pred = tfidf_logit_pipeline.predict(valid_texts)

CPU times: total: 3.83 s
Wall time: 3.91 s


In [14]:
print(f"Точність за даними валідації: {accuracy_score(y_valid, valid_pred): .4f}")

Точність за даними валідації:  0.7210


<span style="color:red; font-size:2em;">Завдання 1</span>

**Для всіх варіантів:**

В Завданні 2 вам потрібно доповнити побудовані на попередньому кроці токенайзер та модель логістичної регресії.

**Для варіантів 1-2:**

Використайте N-грами у векторизаторі TF-IDF і поясніть, як вони працюють.

**Для варіантів 3-4:**

Дослідіть вплив регулярізації L1 (Lasso) і L2 (Ridge) на побудовану модель логістичної регресії. Зробіть висновок щодо впливу певного типу регулярізації (Lasso та Ridge) на результат моделювання.

**Для варіантів 5-6:**

Проведіть експерименти з параметром `max_features` у векторизаторі TF-IDF і зробіть висновок щодо його впливу на кількість ознак.

**Для варіантів 7-8:**

Проведіть експерименти з різними пороговими значеннями для моделі логістичної регресії. Дослідіть, як зміна порогу впливає на частку правильних відповідей, яку генерує побудована модель.
**Для варіантів 9-10:**

Застосуйте k-кратну перехресну перевірку (k-fold cross-validation) до побудованої моделі логістичної регресії та порівняйте результати за тестовим набором. Якими є переваги та недоліки перехресної перевірки.

**Для варіантів 11-12:**

Інтерпретуйте результати моделі логістичної регресії, з огляду на значення коефіцієнтів цієї моделі. Як отримані значення вплинули на результат задачі класифікації?

**Для варіантів 13-14:**

Візуалізуйте та інтерпретуйте криві навчання та валідації побудованої логістичної регресії.

**Для варіантів 15-16:**

Налаштуйте наявні гіперпараметри моделі логістичної регресії за механізмом `GridSearchCV`.

**Для варіантів 17-18:**

Вилучіть та візуалізуйте назви ознак із векторизатора TF-IDF.

**Для варіантів 19-20:**

Проаналізуйте ознаки, які були створені векторизатором TF-IDF. Дослідіть вплив різних слів та їхні відповідні TF-IDF-оцінки.


<a class="anchor" id="lab-4.3"></a>

## <span style="color:blue; font-size:1.2em;">4.3. Інтерпретація та порівняння моделі</span>

[Повернутися до змісту](#lab-4)

In [15]:
# Нижче подамо матрицю невідповідностей для допомоги
def plot_confusion_matrix(
    actual,
    predicted,
    classes,
    normalize=False,
    title="Confusion matrix",
    figsize=(7, 7),
    cmap=plt.cm.Blues,
    path_to_save_fig=None,
):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    import itertools

    from sklearn.metrics import confusion_matrix

    cm = confusion_matrix(actual, predicted).T
    if normalize:
        cm = cm.astype("float") / cm.sum(axis=1)[:, np.newaxis]

    plt.figure(figsize=figsize)
    plt.imshow(cm, interpolation="nearest", cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=90)
    plt.yticks(tick_marks, classes)

    fmt = ".2f" if normalize else "d"
    thresh = cm.max() / 2.0
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(
            j,
            i,
            format(cm[i, j], fmt),
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black",
        )

    plt.tight_layout()
    plt.ylabel("Predicted label")
    plt.xlabel("True label")

    if path_to_save_fig:
        plt.savefig(path_to_save_fig, dpi=300, bbox_inches="tight")

<span style="color:red; font-size:2em;">Завдання 3</span>

**Для всіх варіантів:**

В Завданні 3 вам потрібно інтерпретувати отримані результати щодо передбачення саркастичного тексту.

**Для варіантів 1-2:**



**Для варіантів 3-4:**

Обрахуйте та інтерпретуйте співвідношення шансів для конкретних ознак, з огляду на те як вони пов’язані з ймовірністю того, що текст буде саркастичним.

**Для варіантів 5-6:**

Виконайте візуальний аналіз тих випадків, за яких модель логістичної регресії дала неправильні прогнози, і запропонуйте стратегії для покращення продуктивності моделі на основі цього аналізу.

**Для варіантів 7-8:**

Побудуйте модель дерева рішень на тих самих даних, порівняйте його продуктивність із моделлю логістичної регресії та обговоріть плюси та мінуси кожної моделі.

**Для варіантів 9-10:**



**Для варіантів 11-12:**



**Для варіантів 13-14:**



**Для варіантів 15-16:**

Побудуйте та інтерпретуйте криві навчання та валідації для побудованох моделі логістичної регресії. Проаналізуйте, що ці криві відображають з огляду на продуктивність моделі та її недонавчання/перенавчання.

**Для варіантів 17-18:**

Виконайте візуальний аналіз помилок побудованої моделі. Запропонуйте та реалізуйте стратегії для покращення моделі та переобрахуйте її працездатність.

**Для варіантів 19-20:**

Побудуйте модель лінійної регресії на тих самих даних (перетворюючи цільову змінну, якщо необхідно), порівняйте її ефективність із моделлю логістичної регресії та обговоріть, чому лінійна регресія може бути непридатною для цього завдання.


<a class="anchor" id="lab-4.4"></a>

## <span style="color:blue; font-size:1.2em;">4.4. Розширене вдосконалення моделі</span>

[Повернутися до змісту](#lab-4)

[Optional] За потреби, ви можете розглянути окремі векторизатори TF-IDF для коментарів і субредітів. Ми можемо також дотримуватися pipeline, але тоді процес моделювання стає складнішим, як [у прикладі](https://stackoverflow.com/questions/36731813/computing-separate-tfidf-scores-for-two-different-columns-using-sklearn)

In [16]:
subreddits = train_df["subreddit"]
train_subreddits, valid_subreddits = train_test_split(subreddits, random_state=17)

tf_idf_texts = TfidfVectorizer(ngram_range=(1, 2), max_features=50000, min_df=2)
tf_idf_subreddits = TfidfVectorizer(ngram_range=(1, 1))

<span style="color:red; font-size:2em;">Завдання 3</span>

**Для всіх варіантів:**

В Завданні 3 вам потрібно використати різноманітні програмні механізми для покращення роботи моделі.

**Для варіантів 1-2:**



**Для варіантів 3-4:**

Використайте альтернативні методи подання тексту в числовому форматі, такі як Word2Vec, Doc2Vec або вбудовування. Порівняйте ці подання з TF-IDF і обговоріть їхні переваги та недоліки.

**Для варіантів 5-6:**



**Для варіантів 7-8:**

Побудуйте та інтерпретуйте шлях регулярізації для моделі логістичної регресії. Використайте цей аналіз, щоб вибрати відповідний рівень регулярізації.

**Для варіантів 9-10:**



**Для варіантів 11-12:**

Використайте техніки урізноманітнення текстових даних (data augmentation), щоб збільшити розмір і різноманітність навчальних даних. Обговоріть обрані методи та їхній вплив на роботу моделі.

**Для варіантів 13-14:**

Застосуйте такі механізми обробки текстової інформації, як виправлення орфографії чи нормалізація сленгу. Проаналізуйте їх вплив на працездатність моделі логістичної регресії.

**Для варіантів 15-16:**

Визначайте та відфільтруйте викиди в наборі даних. Обґрунтуйте обрання методу виявлення викидів та проаналізуйте його вплив на модель.

**Для варіантів 17-18:**

Виконайте стратифіковану k-кратну перехресну перевірку (stratified k-fold cross-validation), щоб забезпечити збереження розподілу класів у кожній складці. Проаналізуйте вплив стратифікації на результат моделі.

**Для варіантів 19-20:**

Відкалібруйте результати ймовірностей за допомогою моделі логістичної регресії, використовуючи такі методи, як ізотонічна регресія (isotonic regression) або масштабування Платта (Platt scaling). Проаналізуйте важливість добре відкаліброваних ймовірностей та їх вплив на продуктивність моделі.


<a class="anchor" id="lab-4.5"></a>

## <span style="color:blue; font-size:1.2em;">4.5. Практичне застосування результатів інтелектуального аналізу даних</span>

[Повернутися до змісту](#lab-4)

<span style="color:red; font-size:2em;">Завдання 5</span>

В завданні 5 поміркуйте над застосуванням моделі в реальному світі. Проаналізуйте складнощі, що пов’язані з цією проблемою.

**Для варіанту 1:**

Запропонуйте технічні кроки для розгортання побудованої моделі у виробничому середовищі.

**Для варіанту 2:**

Обговоріть стратегії адаптації моделі до роботи з текстом на різних мовах, з огляду на додаткові виклики та складнощі, що із цим пов’язані.

**Для варіанту 3:**

Запропонуйте стратегії налаштування моделі для постійного навчання та вдосконалення на основі відгуків користувачів або нових даних.

**Для варіанту 4:**

Запропонуйте як налаштувати моніторинг розгорнутої моделі та стратегії постійної підтримки моделі.

**Для варіанту 5:**

Оцініть модель, з огляду на аспекти її продуктивності, користувацького досвіду та етичних міркувань, щоб отримати цілісне уявлення про її застосовність і можливості для вдосконалення.

**Для варіанту 6:**

Запропонуйте стратегії, які могли б забезпечити високу постійну точність моделі, оскільки основні дані змінюються із часом.

**Для варіанту 7:**

Оцініть стійкість моделі до атак зловмисників, коли вхідним текстом навмисно маніпулюють, щоб ввести модель в оману. Проаналізуйте стратегії, щоб зробити модель більш стійкою до таких атак.

**Для варіанту 8:**

Запропонуйте, як можна використати відгуки користувачів для покращення моделі та підвищення її ефективності.

**Для варіанту 9:**

Переконайтеся, що модель і її розгортання відповідають відповідним правовим і регуляторним стандартам, особливо щодо конфіденційності та безпеки даних користувачів.

**Для варіанту 10:**

Запропонуйте нові шляхи для виявлення сарказму в тексті.

**Для варіанту 11:**

Оцініть потенційні упередження в моделі в разі застосування в реальних сценаріях виявлення сарказму та обговоріть стратегії для забезпечення справедливості прийняття рішень.

**Для варіанту 12:**

Обговоріть майбутні тенденції та нові технології, які можуть вплинути на сферу класифікації текстів і виявлення сарказму. Запропонуйте шляхи інтеграції цих досягнень у поточний робочий процес.

**Для варіанту 13:**

Оцініть стійкість моделі до можливих негативних зовнішніх впливів.

**Для варіанту 14:**

Виділіть як мінімум 5 етичних проблем, що пов’язані із застосуванням моделі в реальних сценаріях.

**Для варіанту 15:**

Обговоріть важливість пояснюваності та прозорості моделі, особливо в тих програмних застосунках, де користувачі взаємодіють із моделлю. Запропонуйте шляхи покращення цих аспектів.

**Для варіанту 16:**

Обговоріть, які показники ефективності є найбільш важливими в реальному застосуванні та чому, з огляду на конкретний контекст виявлення сарказму.

**Для варіанту 17:**

Оцініть масштабованість моделі та обговоріть будь-які необхідні механізми оптимізації для ефективної роботи в реальних умовах.

**Для варіанту 18:**

Проаналізуйте потенційні проблеми безпеки, що пов’язані з розгортанням моделі, і запропонуйте стратегії для їхнього вирішення.

**Для варіанту 19:**

Проаналізуйте стратегії роботи з двозначністю та важливість контексту для виявлення сарказму.

**Для варіанту 20:**

Проаналізуйте можливість розширення моделі для включення інших модальностей (наприклад, зображення, аудіо) для виявлення сарказму.
